In [44]:
#Importing necessary libraries

from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")


Data Loading and Initial Exploration

In [2]:
#Initialize Spark Session
spark = SparkSession.builder.appName("FlightDelayPrediction").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/24 20:21:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
#Load the dataset
df = spark.read.csv("../Dataset/Combined_Flights_2022.csv",header=True,inferSchema=True)

In [50]:
#Partitioning the dataset
df = df.repartition(5)

In [51]:
#Display schema and first 10 rows
df.printSchema()
df.show(10)

root
 |-- FlightDate: date (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: boolean (nullable = true)
 |-- Diverted: boolean (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- DepDelayMinutes: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- ArrDelayMinutes: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- CRSElapsedTime: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Marketing_Airline_Network: string (nullable = true)
 |-- Operated_or_Branded_Code_Share_Partners: string (nullable = true)
 |

24/10/24 22:18:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:18:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:18:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:18:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:18:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:18:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:18:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:18:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:18:34 WARN RowBasedKeyValueBatch: Calling spill() on

+----------+--------------------+------+----+---------+--------+----------+-------+---------------+--------+-------+---------------+-------+--------------+-----------------+--------+----+-------+-----+----------+---------+-------------------------+---------------------------------------+------------------------+---------------------------+-------------------------------+-----------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+----------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----------------+---------+-------------+-------------+-------+--------+--------------------+----------+-------+---------+--------+------+----------+--------+--------+------------------+----------+-------------+------------------+
|FlightDate|             Airline|Origin|Dest|Cancelled|Diverted|CRSDepTime|DepTime|DepDelayMinutes|DepDelay|

In [12]:
#Count the total number of rows
row_count = df.count()
print("Total rows:", row_count)

Total rows: 4078318


Data Cleaning and Transformation

In [16]:
from pyspark.sql.functions import col, isnan, when, count

In [36]:
df =df.dropna()

In [38]:
df.count()

3944916

In [39]:
# Remove duplicates
df = df.dropDuplicates()

In [46]:
# Filter invalid rows: For example, remove rows where delays are negative
df = df.filter((df["DepDelayMinutes"]>= 0) & (df["ArrDelayMinutes"]>= 0))

In [47]:
df.show()

24/10/24 21:46:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 21:46:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 21:46:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 21:46:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 21:46:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 21:46:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 21:46:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 21:46:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 21:46:04 WARN RowBasedKeyValueBatch: Calling spill() on

+----------+--------------------+------+----+---------+--------+----------+-------+---------------+--------+-------+---------------+-------+--------------+-----------------+--------+----+-------+-----+----------+---------+-------------------------+---------------------------------------+------------------------+---------------------------+-------------------------------+-----------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+---------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+--------------------+---------+-------------+--------------+-------+--------+--------------------+----------+-------+---------+--------+------+----------+--------+--------+------------------+----------+-------------+------------------+
|FlightDate|             Airline|Origin|Dest|Cancelled|Diverted|CRSDepTime|DepTime|DepDelayMinutes|DepDe

Data Analysis Using Spark SQL

In [48]:
# Register the cleaned DataFrame as a SQL temporary view

df.createOrReplaceTempView("flight_data")

In [49]:
# Aggregation: Calculate summary statistics for delays
spark.sql("""
    SELECT 
        AVG(DepDelayMinutes) AS avg_dep_delay,
        STDDEV(DepDelayMinutes) AS stddev_dep_delay,
        AVG(ArrDelayMinutes) AS avg_arr_delay,
        STDDEV(ArrDelayMinutes) AS stddev_arr_delay
    FROM flight_data
""").show()

24/10/24 22:07:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:07:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:07:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:07:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:07:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:07:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:07:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:07:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/10/24 22:07:39 WARN RowBasedKeyValueBatch: Calling spill() on

+------------------+-----------------+------------------+-----------------+
|     avg_dep_delay| stddev_dep_delay|     avg_arr_delay| stddev_arr_delay|
+------------------+-----------------+------------------+-----------------+
|15.940134086505264|52.16239928825892|15.783071426615928|51.98423581313931|
+------------------+-----------------+------------------+-----------------+

